In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

/kaggle/input/wns-hack-v0/test.csv
/kaggle/input/wns-hack-v0/custom.css
/kaggle/input/wns-hack-v0/train.csv
/kaggle/input/wns-hack-v0/__results__.html
/kaggle/input/wns-hack-v0/__output__.json
/kaggle/input/wns-hack-v0/__notebook__.ipynb
/kaggle/input/av-wns-hack/test_aq1fgdb/test.csv
/kaggle/input/av-wns-hack/train_na17sgz/train.csv
/kaggle/input/av-wns-hack/train_na17sgz/view_log.csv
/kaggle/input/av-wns-hack/train_na17sgz/item_data.csv
/kaggle/input/av-wns-hack/sample_submission_ipsblct/sample_submission.csv


In [2]:
import os, gc

In [3]:
from datetime import datetime
import datetime
def timer(tag_name):
    def timer_decorator(fn):
        def _fn(*args, **kwargs):
            s = datetime.datetime.now()
            output = fn(*args, **kwargs)
            e = datetime.datetime.now()
            print('[{}] {} completed in {}'.format(tag_name, fn.__name__, e - s))
            return output

        return _fn

    return timer_decorator

In [4]:
Path_consol = '../input/wns-hack-v0'
Path_raw = '../input/av-wns-hack/train_na17sgz'
train = pd.read_csv(os.path.join(Path_consol,'train.csv'))
test = pd.read_csv(os.path.join(Path_consol,'test.csv'))
log = pd.read_csv(os.path.join(Path_raw,'view_log.csv'))
item = pd.read_csv(os.path.join(Path_raw,'item_data.csv'))

In [5]:
@timer("Left Merge")
def merge(df1,df2,onCol):
    return pd.merge(df1,df2,on=onCol,how='left')

In [6]:
### merge item details with log
log = merge(log,item,'item_id')
log.head()

[Left Merge] merge completed in 0:00:00.949589


,server_time,device_type,session_id,user_id,item_id,item_price,category_1,category_2,category_3,product_type
0,2018-10-15 08:58:00,android,112333,4557,32970,54685.0,16.0,56.0,253.0,3184.0
1,2018-10-15 08:58:00,android,503590,74788,7640,1376.0,7.0,71.0,228.0,545.0
2,2018-10-15 08:58:00,android,573960,23628,128855,4544.0,4.0,38.0,62.0,5609.0
3,2018-10-15 08:58:00,android,121691,2430,12774,904.0,17.0,39.0,252.0,2740.0
4,2018-10-15 08:58:00,android,218564,19227,28296,2304.0,12.0,57.0,54.0,7422.0


### Next:
- Figure out the closest session for each txn in train wrt. log txns; both before and after? Same user id.  

In [7]:
log.head(1)

,server_time,device_type,session_id,user_id,item_id,item_price,category_1,category_2,category_3,product_type
0,2018-10-15 08:58:00,android,112333,4557,32970,54685.0,16.0,56.0,253.0,3184.0


In [8]:
train.head(1)

,impression_id,impression_time,user_id,app_code,os_version,is_4G,is_click,impression_time_day,impression_time_minute,impression_time_second,LogPresent_24_hour_before,Count_LogPresent_24_hour_before,LogPresent_192_hour_before,Count_LogPresent_192_hour_before,LogPresent_288_hour_before,Count_LogPresent_288_hour_before
0,c4ca4238a0b923820dcc509a6f75849b,2018-11-15 00:00:00,87862,422,old,0,0,15,0,11,0,0,0,0,0,0


In [9]:
from tqdm import tqdm_notebook
import pdb

@timer('merge from log approximately')
def getMergeFromLog(df,log):
    name = "NearestLog_"
    df.loc[:,name+"server_time"] = 0
#     df.loc[:,name+"device_type_0"] = 'Missing'
    df.loc[:,name+"user_id"] = 'Missing'  ### for QC
#     df.loc[:,name+"item_id"] = 'Missing'

    for uID,uDF in tqdm_notebook(df[['user_id','impression_time']].groupby(by='user_id')):
#         pdb.set_trace()
        temp = log.loc[log['user_id']==uID,:]
        adArr = pd.to_datetime(uDF['impression_time'])
        logArr = pd.to_datetime(temp['server_time'])
        for qpt in adArr:
            idxQpt = adArr[adArr==qpt].index
            diff = (qpt - logArr)
            diff.index = logArr.index = temp.index
            nullTD = datetime.timedelta(0)
            if len(diff[diff>nullTD]) >= 1:
                try:
                    cut1,m = diff[diff>nullTD],min(diff[diff>nullTD])
                    mIDx = (cut1[cut1==m].index)
                    potentialLogRows = temp[temp.index.isin(mIDx)]               
                    df.loc[idxQpt,name+"server_time"] = potentialLogRows['server_time'].values[0] ### has to be same as min value is same
                    df.loc[idxQpt,name+"user_id"] = potentialLogRows['user_id'].values[0]  ### for QC, should be same
                    for t in range(len(potentialLogRows['device_type'].values.astype(list))):
                        df.loc[idxQpt,name+"device_type_{}".format(t)] = potentialLogRows['device_type'].values.astype(list)[t]
                        df.loc[idxQpt,name+"item_id_{}".format(t)] = potentialLogRows['item_id'].values.astype(list)[t]
                    #print("Success")
                except:
                    cut1,m = diff[diff>nullTD],min(diff[diff>nullTD])
                    mIDx = (cut1[cut1==m].index)
                    print("earlier",mIDx)
                    print(mIDx)
                    potentialLogRows = temp[temp.index.isin(mIDx)]
                    print(potentialLogRows,"Null diff shape")
    return df

In [10]:
train_new = getMergeFromLog(train,log)
test_new = getMergeFromLog(test,log)

In [11]:
train.head()

,impression_id,impression_time,user_id,app_code,os_version,is_4G,is_click,impression_time_day,impression_time_minute,impression_time_second,...,NearestLog_device_type_9,NearestLog_item_id_9,NearestLog_device_type_10,NearestLog_item_id_10,NearestLog_device_type_11,NearestLog_item_id_11,NearestLog_device_type_12,NearestLog_item_id_12,NearestLog_device_type_13,NearestLog_item_id_13
0,c4ca4238a0b923820dcc509a6f75849b,2018-11-15 00:00:00,87862,422,old,0,0,15,0,11,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,45c48cce2e2d7fbdea1afc51c7c6ad26,2018-11-15 00:01:00,63410,467,latest,1,1,15,1,11,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,70efdf2ec9b086079795c442636b55fb,2018-11-15 00:02:00,71748,259,intermediate,1,0,15,2,11,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,8e296a067a37563370ded05f5a3bf3ec,2018-11-15 00:02:00,69209,244,latest,1,0,15,2,11,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,182be0c5cdcd5072bb1864cdee4d3d6e,2018-11-15 00:02:00,62873,473,latest,0,0,15,2,11,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [12]:
train_new.to_csv('trainWithLogMergeApprox.csv',index=False)
test_new.to_csv('testWithLogMergeApprox.csv',index=False)

/opt/conda/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/opt/conda/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/opt/conda/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/opt/conda/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:519: FutureWarning: Passing (type, 1) or 

### Fin 